In [241]:
## Import Some Libraries
import json
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [242]:
class LoadModel:
    def __init__(self, file_path):
        with open(file_path, 'r') as f:
            model_data = json.load(f)
        
        layers = model_data['layers']
        n_layers = len(layers)
        
        self.weights = []
        self.biases = []
        self.activations = []
        self.input = []
        self.target = []
        self.sigmoid_gradient = lambda x: x * (1. - x)
        self.relu_gradient = lambda x: 1. * (x > 0)
        self.linear_gradient = lambda x: np.ones_like(x)
        self.tanh_gradient = lambda x: 1 - np.tanh(x)**2
        self.softmax_gradient = lambda x: x * (1. - x)
        self.learning_rate = model_data['learning_parameters']['learning_rate']
        self.batch_size = model_data['learning_parameters']['batch_size']
        self.epochs = model_data['learning_parameters']['max_iteration']
        self.error = model_data['learning_parameters']['error_threshold']
        
        
        for i in range(1, n_layers):
            layer = layers[i]
            
            if layer['tipe'] == 'hidden':
                n_neurons = layer['n_neuron']
                weight = np.array(layer['weight']).astype(np.float64)
                bias = np.array(layer['bias']).astype(np.float64)
                activation = layer['activation_function']
                input_layer = layer['input']
                target = layer['target']
                
                self.weights.append(weight)
                self.biases.append(bias)
                self.activations.append(activation)
                self.input.append(input_layer)
                self.target.append(target)
                
            elif layer['tipe'] == 'output':
                n_neurons = layer['n_neuron']
                weight = np.array(layer['weight']).astype(np.float64)
                bias = np.array(layer['bias']).astype(np.float64)
                activation = layer['activation']
                input_layer = layer['input']
                target = layer['target']
                
                self.weights.append(weight)
                self.biases.append(bias)
                self.activations.append(activation)
                self.input.append(input_layer)
                self.target.append(target)
    
    def get_weights(self):
        return self.weights
    
    def get_biases(self):
        return self.biases
    
    def get_input(self):
        return self.input
    
    def get_target(self):
        return self.target
    
    def get_activations(self):
        return self.activations
    
    def get_learning_rate(self):
        return self.learning_rate
    
    def get_batch_size(self):
        return self.batch_size
    
    def get_epochs(self):
        return self.epochs
    
    def get_layer(self, layer):
        return self.weights[layer], self.biases[layer], self.activations[layer]
    
    def get_layer_weights(self, layer):
        return self.weights[layer]
    
    def get_layer_biases(self, layer):
        return self.biases[layer]
    
    def get_layer_activations(self, layer):
        return self.activations[layer]
    
    def get_model(self):
        return self.weights, self.biases, self.activations
    
    def get_activation_gradient(self, activation):
        if activation == 'relu':
            return lambda x: 1. * (x > 0)
        elif activation == 'sigmoid':
            return lambda x: x * (1. - x)
        elif activation == 'linear':
            return lambda x: np.ones_like(x)
        elif activation == 'softmax':
            return lambda x: x * (1. - x)

    
    # print model per layer
    def print_model(self):
        for i in range(len(self.weights)):
            print('Layer', i+1)
            print(self.biases[i])
            print(self.weights[i])
            print(self.activations[i])
            print('')

In [243]:
class FFNN:
    def __init__(self, model_path):
        self.model = LoadModel(model_path)
        self.weights = self.model.get_weights()
        self.biases = self.model.get_biases()
        self.activations = self.model.get_activations()
        self.input = self.model.get_input()
        self.target = self.model.get_target()
        self.learning_rate = self.model.get_learning_rate()
        self.batch_size = self.model.get_batch_size()
        self.epochs = self.model.get_epochs()

    def forward_propagation(self, X):
        input_layer = X
        self.layer_outputs = []  # save outputs of each layer
        n_layers = len(self.weights) + 1
        for i in range(n_layers - 1):
            dot_product = np.dot(input_layer, self.weights[i]) + self.biases[i]
            if self.activations[i] == 'relu':
                output_layer = np.maximum(0, dot_product)
            elif self.activations[i] == 'sigmoid':
                output_layer = 1 / (1 + np.exp(-dot_product))
            elif self.activations[i] == 'linear':
                output_layer = dot_product
            elif self.activations[i] == 'softmax':
                output_layer = np.exp(dot_product) / np.sum(np.exp(dot_product), axis=0)
            input_layer = output_layer
            self.layer_outputs.append(input_layer)

        return output_layer

    def backward_propagation(self, X, y_true, learning_rate):
        n_samples = len(X)
        n_layers = len(self.weights) + 1

        # calculate gradient for the last layer
        y_pred = self.layer_outputs[-1]
        if self.activations[-1] == 'sigmoid': 
            error = (y_pred - y_true) * self.model.sigmoid_gradient(y_pred)
        elif self.activations[-1] == 'relu':
            error = (y_pred - y_true) * self.model.relu_gradient(y_pred)
        elif self.activations[-1] == 'linear':
            error = (y_pred - y_true) * self.model.linear_gradient(y_pred)
        elif self.activations[-1] == 'softmax':
            error = (y_pred - y_true) * self.model.softmax_gradient(y_pred)

        # calculate gradient for the other layers
        for i in range(n_layers - 2, -1, -1):
            X_with_bias = np.concatenate((X, np.tile([1], (X.shape[0], 1)).reshape(X.shape[0], 1)), axis=1)
            gradient = np.dot(error.T, X_with_bias)
            weight_gradient = gradient[:, :-1]
            bias_gradient = gradient[:, -1:]

            self.weights[i] -= learning_rate * weight_gradient.T
            self.biases[i] -= learning_rate * (bias_gradient.T)[0]

            new_m = np.append(self.weights[i], [self.biases[i]], axis=0)
            
            if self.activations[i] == 'sigmoid':
                new_m = np.ones_like(self.layer_outputs[i])
                error = np.dot(error, new_m.T) * self.model.sigmoid_gradient(self.layer_outputs[i])
            elif self.activations[i] == 'relu':
                error = np.dot(error, new_m.T) * self.model.relu_gradient(self.layer_outputs[i])
            elif self.activations[i] == 'linear':
                error = np.dot(error, new_m.T) * self.model.linear_gradient(self.layer_outputs[i])
            elif self.activations[i] == 'softmax':
                error = np.dot(error, self.weights[i].T) * self.model.softmax_gradient(self.layer_outputs[i])

    def train(self, X, y_true, learning_rate, epochs, batch_size):
        for epoch in range(epochs):
            # shuffle the training data
            p = np.random.permutation(len(X))
            X, y_true = X[p], y_true[p]
            

            # split the data into mini-batches
            for i in range(0, len(X), batch_size):
                X_batch, y_batch = X[i:i+batch_size], y_true[i:i+batch_size]

                # forward propagation
                self.forward_propagation(X_batch)

                # backward propagation
                self.backward_propagation(X_batch, y_batch, learning_rate)

            y_pred = self.predict(X)
            loss = np.mean((y_pred - y_true) ** 2)

            if loss < 0.01:
                break


    def predict(self, X):
        if len(X.shape) == 1:
            return self.forward_propagation(X)
        else:
            predictions = []
            for instance in X:
                predictions.append(self.forward_propagation(instance))
            return np.array(predictions)

    def accuracy(self, X, y_true):
        y_pred = self.predict(X)
        y_pred = (y_pred >= 0.5).astype(int)
        return np.mean(y_pred == y_true)

    def print_model(self):
        self.model.print_model()

    def draw_network(self):
        G = nx.DiGraph()

        pos = {}

        # add nodes
        input_nodes = [f'X_{i + 1}' for i in range(len(self.weights[0]))]
        G.add_nodes_from(input_nodes, layer='Input')
        G.add_nodes_from(['b0'], layer='Bias')
        for i in range(len(input_nodes)):   
            pos[f'X_{i + 1}'] = [0, i + 1]
        pos['b0'] = [0, len(self.weights[0]) + 1]

        hidden_nodes = []

        for i in range(len(self.weights) - 1):
            hidden_nodes.append([f'h{i + 1}_{j + 1}' for j in range(len(self.weights[i][0]))])
            G.add_nodes_from(hidden_nodes[i], layer='Hidden')
            G.add_nodes_from([f'b{i + 1}'], layer='Bias')
            for j in range(len(hidden_nodes[i])):
                pos[f'h{i + 1}_{j + 1}'] = [i + 1, j + 1]
            pos[f'b{i + 1}'] = [i + 1, len(self.weights[i][0]) + 1]

        output_nodes = [f'y_{i + 1}' for i in range(len(self.weights[-1][0]))]
        G.add_nodes_from(output_nodes, layer='Output')
        for i in range(len(output_nodes)):
            pos[f'y_{i + 1}'] = [len(self.weights), i + 1]

        # add edges
        for i in range(len(self.weights)):
            for j in range(len(self.weights[i])):
                for k in range(len(self.weights[i][j])):
                    if i == 0:
                        G.add_edge(input_nodes[j], hidden_nodes[i][k], weight=self.weights[i][j][k])
                        G.add_edge(f'b0', hidden_nodes[i][k], weight=self.biases[i][k])
                    elif i == len(self.weights) - 1:
                        G.add_edge(hidden_nodes[i - 1][j], output_nodes[k], weight=self.weights[i][j][k])
                        G.add_edge(f'b{i}', output_nodes[k], weight=self.biases[i][k])
                    else:
                        G.add_edge(hidden_nodes[i - 1][j], hidden_nodes[i][k], weight=self.weights[i][j][k])
                        G.add_edge(f'b{i}', hidden_nodes[i][k], weight=self.biases[i][k])

        # draw network
        edge_labels = nx.get_edge_attributes(G, 'weight')

        nx.draw_networkx_nodes(G, pos, node_size=1500, node_color='lightblue')
        nx.draw_networkx_edges(G, pos, width=2, edge_color='black', arrows=True, arrowsize=25)
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, label_pos=0.60)
        nx.draw_networkx_labels(G, pos, font_size=12, font_family='sans-serif')
        plt.axis('off')
        plt.show()


# main function
def main():
    # load model
    # filename = input("Enter the filename: ")
    filename = 'softmax'
    model_path = filename + '.json'
    model = FFNN(model_path)
    
    # input linear
    X_train = model.input[0]
    Y_train = model.target[0]
    X_train = np.array(X_train)
    Y_train = np.array(Y_train)
    
    # train model
    model.train(X_train, Y_train, model.learning_rate, model.epochs, model.batch_size)

    # print model
    model.print_model()

    # draw network
    # model.draw_network()

if __name__ == '__main__':
    main()

Layer 1
[0.1 0.2]
[[ 0.3861139  -0.48568122]
 [ 0.7138861   0.78568122]]
softmax

